# Reinforcement Learning, 2024-01 
# Tarea 3 - Programación Dinámica

> Daniel Villar González, 201923374.  
> Daniel Alvarez, 201911320.

## **Descripción de tarea**

## **Librerias**

In [2]:
import numpy as np
import random

## **Punto 1**

Considere el MDP que usted formuló para modelar este problema en el taller anterior.
Escriba un módulo de Python que permita la interacción de un agente con el ambiente
para el MDP formulado. Su implementación debe:
- Dada una acción en un estado, retornar la recompensa y el estado resultante en
esa acción. Considere el caso especial del estado terminal.
- Ejecutar una política arbitraria.

In [3]:
def construirMDP(casillasAzules, casillasRojas, p, factorDescuento):
    if factorDescuento > 1 or factorDescuento <= 0:
        return "El valor de descuento no está entre los límites permitidos"
    matrizAdelante = []
    for i in range(100):
        matrizAdelante.append([0] * 100)
    matrizRecompensasAdelante = []
    for i in range(100):
        matrizRecompensasAdelante.append([0] * 100)
    matrizAtras = []
    for i in range(100):
        matrizAtras.append([0] * 100)
    matrizRecompensasAtras= []
    for i in range(100):
        matrizRecompensasAtras.append([0] * 100)
    recompensas = {}
    for i in range(len(casillasAzules)):
        recompensas[casillasAzules[i]] = 1
    for i in range(len(casillasRojas)):
        recompensas[casillasRojas[i]] = -1
    transiciones = {8:26, 21:82, 43:77, 50:91, 54:93, 62:96, 66:87, 80:100, 98:28, 95:24, 92:51, 83:19, 73:1, 69:33, 64:36, 59:17, 55:7, 52:11, 48:9, 46:5, 44:22}
    llaves_transiciones = transiciones.keys()
    llaves_recompensas = recompensas.keys()
    for i in range(len(matrizAdelante)): 
        for j in range(0, 6):
            destino = i + j + 1 
            if destino >= 100:  
                destino = 99 - (destino - 100)  
            if destino+1 in llaves_transiciones:
                matrizAdelante[i][transiciones[destino+1]-1] += p[j]  
                if destino+1 in llaves_recompensas:
                    matrizRecompensasAdelante[i][transiciones[destino+1]-1] += recompensas[destino+1]
            else:
                matrizAdelante[i][destino] += p[j] 
                if destino+1 in llaves_recompensas:
                    matrizRecompensasAdelante[i][destino] += recompensas[destino+1]
    for i in range(len(matrizAtras)): 
        for j in range(0, 6):
            destino = i - j 
            if destino <= 0:  
                destino = abs(destino) + 1
            if destino in llaves_transiciones:
                matrizAtras[i][transiciones[destino]-1] += p[j]  
                if destino in llaves_recompensas:
                    matrizRecompensasAtras[i][transiciones[destino]-1] += recompensas[destino]
            else:
                matrizAtras[i][destino-1] += p[j] 
                if destino in llaves_recompensas:
                    matrizRecompensasAtras[i][destino-1] += recompensas[destino]
    return matrizAdelante, matrizAtras, matrizRecompensasAdelante, matrizRecompensasAtras

In [11]:
class MDPenv:
    def __init__(self, casillasAzules, casillasRojas, p, factorDescuento):
        self.matrizAdelante, self.matrizAtras, self.matrizRecompensasAdelante, self.matrizRecompensasAtras = construirMDP(casillasAzules, casillasRojas, p, factorDescuento)
        self.estado_actual = 0
        self.factorDescuento = factorDescuento
        self.terminal_state = casillasAzules

    def accion(self, movimiento):
        """
        Toma la acción dependiendo de la decisión del agente
        sabemos que si se encuentra en estado terminal la recompensa es 0, y el
        nuevo estado es el mismo.
        """
        for state in self.terminal_state:
            if self.estado_actual == state:
                return 0, self.estado_actual

        # Decidir la matriz de transición y de recompensas a usar basado en la acción
        if movimiento == 'adelante':
            transiciones = self.matrizAdelante[self.estado_actual]
            recompensas = self.matrizRecompensasAdelante[self.estado_actual]
        elif movimiento == 'atras':
            transiciones = self.matrizAtras[self.estado_actual]
            recompensas = self.matrizRecompensasAtras[self.estado_actual]
        # else:
        #     raise ValueError("Movimiento no válido. Use 'adelante' o 'atras'.")
        
        # depende de los valores de la matriz de transicion que se obtiene con p 
        nuevo_estado = self._determinar_nuevo_estado(transiciones)
        # recibe recompensa
        recompensa = recompensas[nuevo_estado]
        # aquí actualizamos el estado :)
        self.estado_actual = nuevo_estado
        # devolvemos la dos cosas que ve tomada la acción recompensa y nuevo estado
        return recompensa, nuevo_estado
    
    # aqui se toma una decisión aleatoria de los siguientes estados en función de la matriz de transición 
    # esto tiene sentido para el agente pero se debe cambiar para SARSA porque parece que no tiene aprendizaje
    def _determinar_nuevo_estado(self, transiciones):
        return random.choices(range(100), weights=transiciones)[0]

    def ejecutar_politica(self, politica, pasos):
        """
        Ejecuta una política dada durante un número especificado de pasos.
        política es una función que toma un estado y devuelve una acción.
        """
        historia = []
        for _ in range(pasos):
            for state in self.terminal_state:
                if self.estado_actual == state:
                    break
            accion = politica(self.estado_actual)
            recompensa, estado = self.accion(accion)
            historia.append((self.estado_actual, accion, recompensa, estado))
        return historia
    
# TEST

def politica_simple(estado):
    return 'adelante' if estado < 50 else 'atras'

mdp = MDPenv([99, 79], [22, 38, 44, 66, 88], [0.1, 0.1, 0.2, 0.3, 0.2, 0.1], 0.99)
historia = mdp.ejecutar_politica(politica_simple, 10)
print(historia)


[(1, 'adelante', 0, 1), (5, 'adelante', 0, 5), (11, 'adelante', 0, 11), (14, 'adelante', 0, 14), (81, 'adelante', 0, 81), (78, 'atras', 1, 78), (74, 'atras', 0, 74), (69, 'atras', 0, 69), (66, 'atras', 0, 66), (60, 'atras', 0, 60)]


## **Punto 2**

## **Punto 3**

In [10]:
class SarsaAgent:
    def __init__(self, num_states, discount_factor=0.99, alpha=0.5, epsilon=0.1):
        self.q_table = np.zeros((num_states, 2))  # acciones
        self.discount_factor = discount_factor
        self.alpha = alpha
        self.epsilon = epsilon

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            return 'adelante' if np.random.rand() < 0.5 else 'atras'  # Acción aleatoria
        else:
            action_index = np.argmax(self.q_table[state])
            return 'adelante' if action_index == 0 else 'atras'  # Mejor acción conocida

    def update_q_table(self, state, action, reward, next_state, next_action):
        action_index = 0 if action == 'adelante' else 1
        next_action_index = 0 if next_action == 'adelante' else 1
        predict = self.q_table[state, action_index]
        target = reward + self.discount_factor * self.q_table[next_state, next_action_index]
        self.q_table[state, action_index] += self.alpha * (target - predict)

def train_sarsa(mdp, agent, num_episodes):
    for episode in range(num_episodes):
        state = mdp.estado_actual
        action = agent.choose_action(state)

        while state != mdp.terminal_state:
            reward, next_state = mdp.accion(action)
            next_action = agent.choose_action(next_state)

            agent.update_q_table(state, action, reward, next_state, next_action)

            state = next_state
            action = next_action

            if state == mdp.terminal_state:
                break

# Instanciación y entrenamiento
mdp = MDPenv([2, 5, 10], [15, 25, 35], [0.1, 0.1, 0.2, 0.3, 0.2, 0.1], 0.99)
num_states = 100
sarsa_agent = SarsaAgent(num_states)
train_sarsa(mdp, sarsa_agent, num_episodes=500)

## **Punto 4**

## **Punto 5**

ambos